In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [39]:
import os
import cv2 
import matplotlib.pyplot as plt
from tqdm import tqdm
from random import shuffle
%matplotlib inline
classes=['ALB','LAG','YFT','SHARK','DOL','BET']
TRAIN_DIR='../input/train/'
IMG_SIZE=500

def prepare_training_data():
    training_data = []
    for img_class in classes:
        for img in tqdm(os.listdir(TRAIN_DIR+img_class)):
            raw_img=cv2.imread(TRAIN_DIR+img_class+'/'+img,cv2.IMREAD_GRAYSCALE)
            r = 100.0 / raw_img.shape[0]
            dim = (250, int(raw_img.shape[1] * r))
            raw_img=cv2.resize(raw_img, dim, interpolation = cv2.INTER_AREA)
            label = label_img(img_class)
            training_data.append([np.array(raw_img),np.array(label)])
        shuffle(training_data)
    #np.save('../input/train/train_data.npy', training_data)
    return training_data

def label_img(img_class):
    if img_class == 'ALB': return [1,0,0,0,0,0]
    elif img_class == 'LAG': return [0,1,0,0,0,0]
    elif img_class == 'YFT': return [0,0,1,0,0,0]
    elif img_class == 'SHARK': return [0,0,0,1,0,0]
    elif img_class == 'DOL': return [0,0,0,0,1,0]
    elif img_class == 'BET': return [0,0,0,0,0,1]

In [40]:
train_data=prepare_training_data()

In [44]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

LR = 1e-3

convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 2, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')